In [14]:
from env import get_connection
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import wrangle as w

import seaborn as sns
from math import sqrt
from scipy import stats

# ignore warnings
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler 
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import QuantileTransformer as qt

# Exercises Plan of Action:
   - Acquire 
   - Prepare
   - Answer the exercises questions

## Acquire the data that we will be using to answer the following questions 

In [7]:
train, val, test= w.wrangle_zillow()

In [8]:
train.head()

,Unnamed: 0,year_built,tax_value,taxamount,bedrooms,bathrooms,area,fips
1567873,1567873,1950.0,191578.0,2712.21,3.0,1.0,1248.0,6037.0
1218138,1218138,1950.0,1345206.0,16539.04,2.0,2.0,1326.0,6037.0
339661,339661,1953.0,356648.0,4575.16,2.0,1.0,1053.0,6037.0
1017133,1017133,1946.0,175069.0,2635.51,3.0,2.0,1256.0,6037.0
40250,40250,1957.0,543000.0,6344.96,3.0,2.0,1640.0,6059.0


In [9]:
train.drop(columns= ['Unnamed: 0'], inplace= True)
val.drop(columns= ['Unnamed: 0'], inplace= True)
test.drop(columns= ['Unnamed: 0'], inplace= True)